<h1>目录<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#预处理" data-toc-modified-id="预处理-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>预处理</a></span></li><li><span><a href="#加载数据和模型" data-toc-modified-id="加载数据和模型-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>加载数据和模型</a></span></li><li><span><a href="#精度，召回率，F和时间随参数的变化" data-toc-modified-id="精度，召回率，F和时间随参数的变化-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>精度，召回率，F和时间随参数的变化</a></span><ul class="toc-item"><li><span><a href="#随MAX_DIS变化" data-toc-modified-id="随MAX_DIS变化-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>随MAX_DIS变化</a></span></li><li><span><a href="#随MAX_ANGLE变化" data-toc-modified-id="随MAX_ANGLE变化-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>随MAX_ANGLE变化</a></span></li><li><span><a href="#随MIN_LEN变化" data-toc-modified-id="随MIN_LEN变化-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>随MIN_LEN变化</a></span></li><li><span><a href="#随MIN_COVER_ANGLE变化" data-toc-modified-id="随MIN_COVER_ANGLE变化-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>随MIN_COVER_ANGLE变化</a></span></li><li><span><a href="#随MIN_COVER_RHO1变化" data-toc-modified-id="随MIN_COVER_RHO1变化-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>随MIN_COVER_RHO1变化</a></span></li><li><span><a href="#随MIN_COVER_RHO2变化" data-toc-modified-id="随MIN_COVER_RHO2变化-3.6"><span class="toc-item-num">3.6&nbsp;&nbsp;</span>随MIN_COVER_RHO2变化</a></span></li></ul></li></ul></div>

# 预处理

In [1]:
%matplotlib inline
dataset = '02_prasad'
raw_path = f'/home/liucc/Workspace/02_Ellipse/03_results/{dataset}/01_raw/'
label_path = f'/home/liucc/Workspace/02_Ellipse/03_results/{dataset}/04_ellipse_label/'

In [2]:
import os 
fnames = os.listdir(raw_path)
fnames = [n[:-4] for n in fnames if n[-3:]=='jpg']
fnames.sort()
print(fnames[:10])

['001', '002', '003', '004', '005', '006', '007', '008', '009', '010']


# 加载数据和模型

In [3]:
import sys
sys.path.append('/home/liucc/Workspace/02_Ellipse/01_data')
from cc_process_image import *
import imageio

raws = get_all_images(f'{raw_path}',fnames,'jpg',itype='gray')

print(len(raws))

/home/liucc/anaconda3/lib/python3.6/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
/home/liucc/anaconda3/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:756: UserWarning: Corrupt EXIF data.  Expecting to read 12 bytes but only got 6. 
  warnings.warn(str(msg))


198


In [4]:
gts = []

for name in fnames:
    lname = label_path+name+'.txt'
    
    with open(lname,'r') as f:
        txt = f.read()
        
        ells_t = [[float(v) for v in l.split(' ')] for l in txt.split('\n')]
        
        gts.append(ells_t)
        

print(len(gts))

198


# 精度，召回率，F和时间随参数的变化

In [5]:
import sys
sys.path.append('/home/liucc/Workspace/02_Ellipse/02_algorithm/02_ellipse_detect')
from cc_ellipse_v2 import *
sys.path.append('../')
from experiment import *

In [13]:
import time
from skimage.feature import canny
def one_task(glob):
    start = time.time()

    prs = []
    
    for im in raws:
        im = preprocess_image(im)
        imo = canny(im, sigma=2)
        ells = detect_ellipses_im(imo,glob)
        
        prs.append(ells)

    end = time.time()
    
    costT = (end-start)/len(raws)
    
    precision,recall,F1,_,_ = cal_metrics(prs,gts,threshold=0.8)
    
    
    return [precision,recall,F1,costT]

In [27]:
#全局参数
GLOB = {
    'MAX_FIT_DIS':2,
    'MAX_DIS':110,
    'MAX_ANGLE':50,
    'MIN_LEN' : 50,
    'MIN_COVER_ANGLE' : 120,
    'MIN_COVER_RHO1' : 0.4,
    'MIN_COVER_RHO2' : 0.3,
}

precision,recall,F1,costT = one_task(GLOB)

/home/liucc/Workspace/02_Ellipse/02_algorithm/02_ellipse_detect/cc_ellipse_v2.py:555: RuntimeWarning: divide by zero encountered in double_scalars
  MA = np.sqrt(FA1/(A+C+FA2)) #长轴
/home/liucc/Workspace/02_Ellipse/02_algorithm/02_ellipse_detect/cc_ellipse_v2.py:555: RuntimeWarning: invalid value encountered in sqrt
  MA = np.sqrt(FA1/(A+C+FA2)) #长轴


In [28]:
print('Precision:%.4f' % precision)
print('Recall:   %.4f' % recall)
print('F1 Score: %.4f' % F1)
print('Avg Time: %.0f' % (costT*1000))

Precision:0.6841
Recall:   0.3684
F1 Score: 0.4789
Avg Time: 470


## 随MAX_DIS变化

In [17]:
from matplotlib.font_manager import FontProperties
fp1 = FontProperties(family="DejaVu Sans",size=20)
fp2 = FontProperties(family="DejaVu Sans",size=18)

GLOB = {
    'MAX_FIT_DIS':2,
    'MAX_DIS':110,
    'MAX_ANGLE':60,
    'MIN_LEN' : 40,
    'MIN_COVER_ANGLE' : 120,
    'MIN_COVER_RHO1' : 0.4,
    'MIN_COVER_RHO2' : 0.33,
}

res01 = []
xs01 = list(range(80,130,10))
for x in xs01:
    GLOB['MAX_DIS'] = x
    res = one_task(GLOB)
    res01.append(res)
    print(f'{x}:{res}')

/home/liucc/Workspace/02_Ellipse/02_algorithm/02_ellipse_detect/cc_ellipse_v2.py:555: RuntimeWarning: divide by zero encountered in double_scalars
  MA = np.sqrt(FA1/(A+C+FA2)) #长轴
/home/liucc/Workspace/02_Ellipse/02_algorithm/02_ellipse_detect/cc_ellipse_v2.py:555: RuntimeWarning: invalid value encountered in sqrt
  MA = np.sqrt(FA1/(A+C+FA2)) #长轴


80:[0.717206132879046, 0.3598290598290598, 0.479225953329539, 0.6028242376115587]
90:[0.7206132879045997, 0.36153846153846153, 0.48150256118383605, 0.5999690125686954]
100:[0.7235494880546075, 0.3623931623931624, 0.48291571753986334, 0.6183213067777229]
110:[0.726027397260274, 0.3623931623931624, 0.48346636259977194, 0.6416568105871027]
120:[0.726027397260274, 0.3623931623931624, 0.48346636259977194, 0.5989958132156218]


## 随MAX_ANGLE变化

In [ ]:
from matplotlib.font_manager import FontProperties
fp1 = FontProperties(family="DejaVu Sans",size=20)
fp2 = FontProperties(family="DejaVu Sans",size=18)

GLOB = {
    'MAX_FIT_DIS':2,
    'MAX_DIS':110,
    'MAX_ANGLE':50,
    'MIN_LEN' : 40,
    'MIN_COVER_ANGLE' : 120,
    'MIN_COVER_RHO1' : 0.4,
    'MIN_COVER_RHO2' : 0.33,
}
res02 = []
xs02 = list(range(40,90,10))
for x in xs02:
    GLOB['MAX_ANGLE'] = x
    res = one_task(GLOB)
    res02.append(res)
    print(f'{x}:{res}')

40:[0.7132867132867133, 0.3487179487179487, 0.4684270952927669, 0.5632547874643345]


/home/liucc/Workspace/02_Ellipse/02_algorithm/02_ellipse_detect/cc_ellipse_v2.py:555: RuntimeWarning: divide by zero encountered in double_scalars
  MA = np.sqrt(FA1/(A+C+FA2)) #长轴
/home/liucc/Workspace/02_Ellipse/02_algorithm/02_ellipse_detect/cc_ellipse_v2.py:555: RuntimeWarning: invalid value encountered in sqrt
  MA = np.sqrt(FA1/(A+C+FA2)) #长轴


50:[0.7282051282051282, 0.3641025641025641, 0.4854700854700855, 0.6006443079071816]
60:[0.726027397260274, 0.3623931623931624, 0.48346636259977194, 0.600905898845557]
70:[0.7078464106844741, 0.3623931623931624, 0.4793668739400791, 0.6552718492469403]


## 随MIN_LEN变化

In [23]:
from matplotlib.font_manager import FontProperties
fp1 = FontProperties(family="DejaVu Sans",size=20)
fp2 = FontProperties(family="DejaVu Sans",size=18)

GLOB = {
    'MAX_FIT_DIS':2,
    'MAX_DIS':110,
    'MAX_ANGLE':50,
    'MIN_LEN' : 50,
    'MIN_COVER_ANGLE' : 120,
    'MIN_COVER_RHO1' : 0.4,
    'MIN_COVER_RHO2' : 0.33,
}


res1 = []
xs1 = list(range(40,80,10))
for x in xs1:
    GLOB['MIN_LEN'] = x
    res = one_task(GLOB)
    res1.append(res)
    print(f'{x}:{res}')

/home/liucc/Workspace/02_Ellipse/02_algorithm/02_ellipse_detect/cc_ellipse_v2.py:555: RuntimeWarning: divide by zero encountered in double_scalars
  MA = np.sqrt(FA1/(A+C+FA2)) #长轴
/home/liucc/Workspace/02_Ellipse/02_algorithm/02_ellipse_detect/cc_ellipse_v2.py:555: RuntimeWarning: invalid value encountered in sqrt
  MA = np.sqrt(FA1/(A+C+FA2)) #长轴


40:[0.7282051282051282, 0.3641025641025641, 0.4854700854700855, 0.5748334051382662]
50:[0.7421602787456446, 0.3641025641025641, 0.48853211009174313, 0.5985291979529641]
60:[0.7544802867383512, 0.3598290598290598, 0.4872685185185185, 0.5806128436868842]
70:[0.779245283018868, 0.352991452991453, 0.4858823529411765, 0.4630034186623313]


## 随MIN_COVER_ANGLE变化

In [24]:
GLOB = {
    'MAX_FIT_DIS':2,
    'MAX_DIS':110,
    'MAX_ANGLE':50,
    'MIN_LEN' : 50,
    'MIN_COVER_ANGLE' : 120,
    'MIN_COVER_RHO1' : 0.4,
    'MIN_COVER_RHO2' : 0.33,
}


res2 = []
xs2 = np.arange(100,140,10)
for x in xs2:
    GLOB['MIN_COVER_ANGLE'] = x
    res = one_task(GLOB)
    res2.append(res)
    print(f'{x}:{res}')

/home/liucc/Workspace/02_Ellipse/02_algorithm/02_ellipse_detect/cc_ellipse_v2.py:555: RuntimeWarning: divide by zero encountered in double_scalars
  MA = np.sqrt(FA1/(A+C+FA2)) #长轴
/home/liucc/Workspace/02_Ellipse/02_algorithm/02_ellipse_detect/cc_ellipse_v2.py:555: RuntimeWarning: invalid value encountered in sqrt
  MA = np.sqrt(FA1/(A+C+FA2)) #长轴


100:[0.7314974182444062, 0.36324786324786323, 0.4854368932038835, 0.5750556281118682]
110:[0.7365684575389948, 0.36324786324786323, 0.4865483686319404, 0.6017270509642784]
120:[0.7421602787456446, 0.3641025641025641, 0.48853211009174313, 0.6054651809461189]
130:[0.7442273534635879, 0.3581196581196581, 0.4835545297172533, 0.5268654787179196]


## 随MIN_COVER_RHO1变化

In [25]:
GLOB = {
    'MAX_FIT_DIS':2,
    'MAX_DIS':110,
    'MAX_ANGLE':50,
    'MIN_LEN' : 50,
    'MIN_COVER_ANGLE' : 120,
    'MIN_COVER_RHO1' : 0.4,
    'MIN_COVER_RHO2' : 0.33,
}


res3 = []
xs3 = np.arange(0.2,0.6,0.05)
for x in xs3:
    GLOB['MIN_COVER_RHO1'] = x
    res = one_task(GLOB)
    res3.append(res)
    print(f'{x}:{res}')

/home/liucc/Workspace/02_Ellipse/02_algorithm/02_ellipse_detect/cc_ellipse_v2.py:555: RuntimeWarning: divide by zero encountered in double_scalars
  MA = np.sqrt(FA1/(A+C+FA2)) #长轴
/home/liucc/Workspace/02_Ellipse/02_algorithm/02_ellipse_detect/cc_ellipse_v2.py:555: RuntimeWarning: invalid value encountered in sqrt
  MA = np.sqrt(FA1/(A+C+FA2)) #长轴


0.2:[0.7408695652173913, 0.3641025641025641, 0.48825214899713465, 0.5734309812988898]
0.25:[0.7408695652173913, 0.3641025641025641, 0.48825214899713465, 0.5790991458025846]
0.3:[0.7408695652173913, 0.3641025641025641, 0.48825214899713465, 0.5662287810836175]
0.35:[0.7408695652173913, 0.3641025641025641, 0.48825214899713465, 0.47119136049289895]
0.39999999999999997:[0.7421602787456446, 0.3641025641025641, 0.48853211009174313, 0.5386072204570578]
0.44999999999999996:[0.7473498233215548, 0.36153846153846153, 0.48732718894009214, 0.539556951233835]
0.49999999999999994:[0.7564575645756457, 0.3504273504273504, 0.4789719626168225, 0.5568899903634582]
0.5499999999999999:[0.7606177606177607, 0.3367521367521368, 0.466824644549763, 0.5837658342688975]


## 随MIN_COVER_RHO2变化

In [26]:
GLOB = {
    'MAX_FIT_DIS':2,
    'MAX_DIS':110,
    'MAX_ANGLE':50,
    'MIN_LEN' : 50,
    'MIN_COVER_ANGLE' : 120,
    'MIN_COVER_RHO1' : 0.4,
    'MIN_COVER_RHO2' : 0.3,
}

res4 = []
xs4 = np.arange(0.2,0.5,0.05)
for x in xs4:
    GLOB['MIN_COVER_RHO2'] = x
    res = one_task(GLOB)
    res4.append(res)
    print(f'{x}:{res}')

/home/liucc/Workspace/02_Ellipse/02_algorithm/02_ellipse_detect/cc_ellipse_v2.py:555: RuntimeWarning: divide by zero encountered in double_scalars
  MA = np.sqrt(FA1/(A+C+FA2)) #长轴
/home/liucc/Workspace/02_Ellipse/02_algorithm/02_ellipse_detect/cc_ellipse_v2.py:555: RuntimeWarning: invalid value encountered in sqrt
  MA = np.sqrt(FA1/(A+C+FA2)) #长轴


0.2:[0.503919372900336, 0.38461538461538464, 0.43625787687833256, 0.5995694025598391]
0.25:[0.6005434782608695, 0.37777777777777777, 0.4637985309548793, 0.5337950978616272]
0.3:[0.6841269841269841, 0.3683760683760684, 0.4788888888888889, 0.48313419264976426]
0.35:[0.7645951035781544, 0.347008547008547, 0.4773662551440328, 0.4688421307188092]
0.39999999999999997:[0.8226164079822617, 0.3170940170940171, 0.45774213448488593, 0.4695673735454829]
0.44999999999999996:[0.875, 0.25726495726495724, 0.39762219286657857, 0.46969818466841573]
